In [2]:
# ==============================================================================
# PART 1: SYSTEM SETUP (LAYOUT-AWARE)
# ==============================================================================

# ... (All imports and configuration are the same) ...
import os
import re
import json
import uuid
import torch
import threading
from typing import List, TypedDict, Optional, Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

from langgraph.graph import StateGraph, END
from llama_cpp import Llama
from transformers import AutoTokenizer, AutoModelForCausalLM
from unstructured.partition.pdf import partition_pdf
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt, wait_exponential
# We need more from docx for advanced table/column layout
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

print("Libraries imported successfully.")

# ... (Configuration, Model Loading, Helper Functions, Glossary, ContextManager, and State are all the same) ...
# NOTE: No changes are needed in sections 2, 3, 4, or 5 of the previous code.
# The core components are already correct. The only change is how we use them.
# For brevity, I am omitting the unchanged sections. They should be copied from the previous final version.
# ==============================================================================
# 2. CONFIGURATION (WITH MEMORY FIXES)
# ==============================================================================
# --- File and Processing Configuration ---
ENGLISH_PDF_PATH = r"C:\Users\sripa\Downloads\Annual Report 2024 Allianz Groupeng.pdf"

# --- MEMORY-SAVING FIXES ARE HERE ---
MAX_ELEMENTS_TO_PROCESS = 15 # <<< FIX #1: Process a smaller batch first to ensure success.
NUM_WORKERS = 1              # <<< FIX #2: The most important change. Process one element at a time to prevent memory overload.

# --- Model Paths & Names ---
REVISER_LLM_PATH = r"C:\Users\sripa\Downloads\Meta-Llama-3.1-8B-Instruct.i1-Q5_K_M.gguf"
TRANSLATOR_MODEL_NAME = "LinguaCustodia/FinTranslate-410M"

# ==============================================================================
# 3. HYBRID MODEL LOADING
# ==============================================================================
print(f"Loading local GGUF model for Reviser/Critic: {REVISER_LLM_PATH}")
try:
    reviser_llm = Llama(model_path=REVISER_LLM_PATH, n_gpu_layers=0, n_ctx=8192, verbose=False, chat_format="llama-3")
    print("Reviser/Critic LLM (Llama-3.1) loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Failed to load local GGUF model: {e}")

print(f"Loading specialist translation model: {TRANSLATOR_MODEL_NAME}")
try:
    fin_tokenizer = AutoTokenizer.from_pretrained(TRANSLATOR_MODEL_NAME)
    fin_translator_model = AutoModelForCausalLM.from_pretrained(TRANSLATOR_MODEL_NAME)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    fin_translator_model.to(DEVICE)
    fin_translator_model.eval()
    print(f"Specialist Translation Model (FinTranslate) loaded successfully on {DEVICE}.")
except Exception as e:
    raise RuntimeError(f"Failed to load translation model from Hugging Face: {e}")

# ==============================================================================
# 4. LINGUACUSTODIA HELPER FUNCTIONS
# ==============================================================================
LANGUAGES = ["en", "de", "es", "fr", "it", "nl", "sv", "pt"]
DOMAINS = {"Annual report": "ar", "General": "general"}

def format_input_for_translator(src: str, tgt_lang: str = "de", src_lang: str = "en", domain: str = "Annual report") -> str:
    domain_code = DOMAINS.get(domain, "general")
    return f"<eos>{src}</src><lang_{tgt_lang}><lang_{src_lang}><dom_{domain_code}>"

# ==============================================================================
# 5. GLOSSARY, STATE, AND FULLY-FEATURED CONTEXT MANAGER
# ==============================================================================
FINANCIAL_GLOSSARY = {
    "Actions": "Maßnahmen", "Annual Report": "Geschäftsbericht", "Asset managers": "Vermögensverwalter",
    "Balance Sheet": "Bilanz", "Basis for preparation": "Grundlagen für die Erstellung",
    "Board of Management": "Vorstand", "Climate change": "Klimawandel",
    "Consolidated financial statements": "Konzernabschluss", "Financial year": "Geschäftsjahr",
    "Revenue": "Umsatz", "Sustainability Statement": "Nachhaltigkeitserklärung"
}

def format_number_de(text: str) -> str:
    return re.sub(r'(\d{1,3}(?:,\d{3})*(\.\d+)?)|(\d+)',
                  lambda m: m.group(0).replace(',', 'X').replace('.', ',').replace('X', '.'),
                  text)

def extract_text_from_html(html: str) -> str:
    if not html: return ""
    return " ".join(BeautifulSoup(html, "lxml").get_text().split())
def extract_text_from_html(html: str) -> str:
    if not html: return ""
    return " ".join(BeautifulSoup(html, "lxml").get_text().split())

class DocumentContextManager:
    def __init__(self):
        self.term_decisions = {}
        # The lock is still good practice even with one worker, in case it's increased later
        self.lock = threading.Lock()
        print("Thread-safe Document Context Manager initialized.")
    def register_decision(self, en_term: str, de_term: str):
        with self.lock:
            en_term_lower = en_term.lower()
            if en_term_lower not in self.term_decisions:
                self.term_decisions[en_term_lower] = de_term
                print(f"CONTEXT MGR: Registered '{en_term}' -> '{de_term}'")
    def get_decision(self, en_term: str) -> Optional[str]:
        with self.lock:
            return self.term_decisions.get(en_term.lower())

class TranslationState(TypedDict):
    current_element: dict
    draft_translation: str
    critiques: List[str]
    final_translation: Optional[str]
    doc_context_manager: DocumentContextManager
    new_decision: Optional[Dict[str, str]]

print("Graph state defined.")

Libraries imported successfully.
Loading local GGUF model for Reviser/Critic: C:\Users\sripa\Downloads\Meta-Llama-3.1-8B-Instruct.i1-Q5_K_M.gguf


llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


Reviser/Critic LLM (Llama-3.1) loaded successfully.
Loading specialist translation model: LinguaCustodia/FinTranslate-410M
Specialist Translation Model (FinTranslate) loaded successfully on cpu.
Graph state defined.


In [4]:
# ==============================================================================
# PART 2: AGENTIC WORKFLOW WITH ALL ENHANCEMENTS (FINAL CORRECTED)
# ==============================================================================

# 6. LANGGRAPH AGENT NODES (WITH ALL FEATURES)
# ==============================================================================

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def initial_translator_node(state: TranslationState) -> dict:
    """Uses the specialist FinTranslate model with its required custom input format."""
    print("\n--- 📝 DRAFT TRANSLATOR (FinTranslate Specialist) ---")
    element = state["current_element"]
    source_text = element.get("text", "")
    if element.get("type") == "Table":
        source_text = extract_text_from_html(element.get("metadata", {}, {}).get("text_as_html", ""))
    if not source_text.strip(): return {"draft_translation": ""}
    
    formatted_text = format_input_for_translator(source_text)

    with torch.no_grad():
        inputs = fin_tokenizer(formatted_text, return_tensors="pt", return_token_type_ids=False).to(DEVICE)
        outputs = fin_translator_model.generate(**inputs, max_new_tokens=1024)
        input_size = inputs["input_ids"].size(1)
        draft = fin_tokenizer.decode(outputs[0, input_size:], skip_special_tokens=True).strip()

    print(f"Draft: '{draft[:120]}...'")
    return {"draft_translation": draft}
def rule_based_reviser_node(state: TranslationState) -> dict:
    """Tier 1: Handles simple, deterministic fixes without an LLM call."""
    print("--- ⚙️ RULE-BASED REVISER (Efficient Fixes) ---")
    draft = state["draft_translation"]
    revised_text = draft
    if any("Formatting Violation" in c for c in state["critiques"]):
        print("  > Applying German number formatting...")
        revised_text = format_number_de(revised_text)
    return {"final_translation": revised_text}

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def llm_reviser_agent_node(state: TranslationState) -> dict:
    """Uses the powerful Llama 3.1 model for complex revisions and dynamic term registration."""
    print("--- 🛠️ LLM REVISER AGENT (Llama-3.1 Generalist) ---")
    source_text, draft_translation, critiques = state["current_element"]["text"], state["draft_translation"], state["critiques"]
    system_prompt = """You are an expert financial editor. Correct the flawed German translation based only on the critiques.
    RULES:
1. Your primary output is ONLY the final, corrected German text. No introductions or markdown.
2. If you make a new, important terminology choice not from the glossary, output it on a new line after the translation in the format: `DECISION::[english_term]::[german_term]`"""
    user_prompt = f"""**Source Text (English):**
"{source_text}"
**Flawed Draft (German):**
"{draft_translation}"
**Mandatory Critiques to Address:**
- {"- ".join(critiques)}"""
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
    response = reviser_llm.create_chat_completion(messages=messages, max_tokens=1024, temperature=0.1)
    full_response = response['choices'][0]['message']['content'].strip()
    final_translation, new_decision = full_response, None
    if "DECISION::" in full_response:
        lines = full_response.splitlines()
        final_translation = "\n".join([line for line in lines if not line.startswith("DECISION::")])
        decision_line = next((line for line in lines if line.startswith("DECISION::")), None)
        if decision_line:
            parts = decision_line.split("::")
            if len(parts) == 3: new_decision = {"en": parts[1].strip(), "de": parts[2].strip()}
    cleaned_translation = re.sub(r"^\s*\*\*.*?\*\*\s*:?\s*", "", final_translation).strip()
    print(f"Revised Translation: '{cleaned_translation[:120]}...'")
    if new_decision: print(f"Reviser proposed new term: {new_decision['en']} -> {new_decision['de']}")
    return {"final_translation": cleaned_translation, "new_decision": new_decision}

# ==============================================================================
# 7. FULL CRITIC SUITE AND RECONSTRUCTOR
# ==============================================================================
def _critique_terminology(source_text: str, draft: str) -> List[str]:
    critiques = []
    for en_term, de_term in FINANCIAL_GLOSSARY.items():
        if re.search(r'\b' + re.escape(en_term) + r'\b', source_text, re.IGNORECASE):
            if not re.search(r'\b' + re.escape(de_term) + r'\b', draft, re.IGNORECASE):
                critiques.append(f"Glossary Violation: '{en_term}' should be '{de_term}'.")
    return critiques

def _critique_formatting(draft: str) -> List[str]:
    return ["Formatting Violation: Numbers may be in US format."] if re.search(r'\b\d{1,3}(,\d{3})+(\.\d+)?\b', draft) else []

def _critique_consistency(source_text: str, draft: str, manager: DocumentContextManager) -> List[str]:
    critiques = []
    for en_term in manager.term_decisions.keys():
        if re.search(r'\b' + re.escape(en_term) + r'\b', source_text, re.IGNORECASE):
            established_translation = manager.get_decision(en_term)
            if established_translation and not re.search(r'\b' + re.escape(established_translation) + r'\b', draft, re.IGNORECASE):
                critiques.append(f"Consistency Violation: '{en_term}' was previously set as '{established_translation}'.")
    return critiques

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def _critique_style(draft: str) -> List[str]:
    if not draft or len(draft.split()) < 3: return []
    messages = [{"role": "system", "content": "You are a German language style checker. Analyze for awkward phrasing or unnatural tone. If an issue exists, describe it in one short sentence. If good, respond 'OK'."}, {"role": "user", "content": draft}]
    response = reviser_llm.create_chat_completion(messages=messages, max_tokens=100, temperature=0.1)
    critique = response['choices'][0]['message']['content'].strip()
    return [] if "OK" in critique or not critique else [f"Stylistic Violation: {critique}"]
def master_critique_node(state: TranslationState) -> dict:
    print("--- 🧐 MASTER CRITIQUE AGENT ---")
    source_text, draft, manager = state["current_element"].get("text", ""), state.get("draft_translation", ""), state["doc_context_manager"]
    critiques = _critique_terminology(source_text, draft) + _critique_formatting(draft) + _critique_consistency(source_text, draft, manager) + _critique_style(draft)
    if critiques:
        print("--- Critiques Found: ---")
        for c in critiques: print(f"  - {c}")
    else:
        print("--- No issues found. Approved. ---")
    return {"critiques": critiques}

def final_reconstructor_node(state: TranslationState) -> dict:
    print("--- ✅ FINAL RECONSTRUCTOR & MEMORY UPDATE ---")
    final_text, source_text, manager = state.get("final_translation") or state.get("draft_translation", ""), state["current_element"]["text"], state["doc_context_manager"]
    new_decision = state.get("new_decision")
    if new_decision: manager.register_decision(new_decision['en'], new_decision['de'])
    for en_term, de_term in FINANCIAL_GLOSSARY.items():
        if re.search(r'\b' + re.escape(en_term) + r'\b', source_text, re.IGNORECASE) and re.search(r'\b' + re.escape(de_term) + r'\b', final_text, re.IGNORECASE):
            manager.register_decision(en_term, de_term)
    final_element = state["current_element"].copy()
    final_element.update({"original_text": source_text, "translated_text": final_text, "critiques_found": state.get("critiques", [])})
    print("-> Element processed and memory updated.")
    return {"final_translation": final_element}

# ==============================================================================
# 8. GRAPH CONSTRUCTION AND EXECUTION
# ==============================================================================
def build_graph():
    workflow = StateGraph(TranslationState)
    nodes = {"translator": initial_translator_node, "master_critic": master_critique_node, "rule_based_reviser": rule_based_reviser_node, "llm_reviser": llm_reviser_agent_node, "reconstructor": final_reconstructor_node}
    for name, node in nodes.items(): workflow.add_node(name, node)
    workflow.set_entry_point("translator")
    workflow.add_edge("translator", "master_critic")
    workflow.add_edge("rule_based_reviser", "reconstructor")
    workflow.add_edge("llm_reviser", "reconstructor")
    def route_after_critique(state: TranslationState) -> str:
        critiques = state.get("critiques", [])
        if not critiques: return "reconstructor"
        return "rule_based_reviser" if {c.split(":")[0] for c in critiques}.issubset({"Formatting Violation"}) else "llm_reviser"
    workflow.add_conditional_edges("master_critic", route_after_critique, {"reconstructor": "reconstructor", "rule_based_reviser": "rule_based_reviser", "llm_reviser": "llm_reviser"})
    workflow.add_edge("reconstructor", END)
    print("LangGraph workflow built successfully with all advanced features.")
    return workflow.compile()

def document_parser(path: str) -> List[dict]:
    print("--- 📄 PARSING DOCUMENT ---")
    if not os.path.exists(path): raise FileNotFoundError(f"File not found: {path}")
    elements = partition_pdf(filename=path, strategy="hi_res", infer_table_structure=True, model_name="yolox", languages=['eng'])
    print(f"Document parsed into {len(elements)} elements.")
    return [el.to_dict() for el in elements]
def create_review_document(filename: str, processed_elements: List[dict]):
    doc = Document()
    doc.add_heading('Financial Translation Review', 0)
    for element in processed_elements:
        if not element: continue
        doc.add_heading(f"Element Type: {element.get('type', 'N/A')}", level=2)
        table = doc.add_table(rows=1, cols=2); table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells; hdr_cells[0].text = 'Original (English)'; hdr_cells[1].text = 'Translated (German)'
        row_cells = table.add_row().cells; row_cells[0].text = element.get('original_text', ''); row_cells[1].text = element.get('translated_text', '')
        if element.get("critiques_found"):
            doc.add_paragraph("Critiques Found:", style='Intense Quote')
            for critique in element["critiques_found"]: doc.add_paragraph(critique, style='List Bullet')
        doc.add_page_break()
    doc.save(filename)
    print(f"\nReview document saved to '{filename}'")

def process_element_task(args):
    """Worker function for parallel/sequential processing."""
    i, element, app, doc_manager = args
    print("="*60 + f"\nProcessing element {i+1} | Type: '{element.get('type', 'UncategorizedText')}'\n" + "="*60)
    if element.get("type", "UncategorizedText") not in ["NarrativeText", "Title", "ListItem", "Table"]:
        skipped_element = element.copy()
        skipped_element["translated_text"] = f"SKIPPED: Type '{element.get('type')}'"
        skipped_element["original_text"] = element["text"]
        return i, skipped_element
    initial_state = {"current_element": element, "doc_context_manager": doc_manager}
    try:
        final_state = app.invoke(initial_state)
        return i, final_state.get('final_translation')
    except Exception as e:
        error_element = element.copy()
        error_element["translated_text"] = f"ERROR: {e}"
        error_element["original_text"] = element["text"]
        return i, error_element

# ==============================================================================
# PART 2: AGENTIC WORKFLOW WITH LAYOUT-AWARE I/O
# ==============================================================================

# --- No changes to the Agent Nodes (6), Critic Suite (7), or Graph Builder (8) ---
# The core translation engine remains the same.
# We are only changing Section 9: The I/O and Execution Logic.
# For brevity, I am omitting the unchanged sections (6, 7, 8). They should be copied from the previous final version.

# ==============================================================================
# 9. ADVANCED DOCUMENT PARSING AND RECONSTRUCTION
# ==============================================================================

def document_parser(path: str) -> Dict[int, List[dict]]:
    """
    Parses the PDF and returns elements grouped by page number.
    This is crucial for page-by-page layout analysis.
    """
    print("--- 📄 PARSING DOCUMENT (Layout-Aware) ---")
    if not os.path.exists(path): raise FileNotFoundError(f"File not found: {path}")
    
    # The 'coordinates' metadata is essential for layout detection
    elements = partition_pdf(filename=path, strategy="hi_res", infer_table_structure=True, model_name="yolox", languages=['eng'], include_page_breaks=True, extract_element_types=True)
    
    paged_elements = {}
    current_page = 1
    for el in elements:
        if el.category == "PageBreak":
            current_page += 1
            continue
        
        if current_page not in paged_elements:
            paged_elements[current_page] = []
            
        # We must store the element as a dict to add our own metadata
        el_dict = el.to_dict()
        paged_elements[current_page].append(el_dict)

    print(f"Document parsed into {len(paged_elements)} pages.")
    return paged_elements

def group_elements_by_column(page_elements: List[dict], page_width: float, full_width_threshold=0.7) -> Dict[str, List[dict]]:
    """
    Takes a list of elements from a single page and groups them into
    'full_width', 'left_column', and 'right_column'.
    """
    columns = {"full_width": [], "left_column": [], "right_column": []}
    page_midpoint = page_width / 2

    for el in page_elements:
        coords = el.get("metadata", {}).get("coordinates")
        if not coords:
            columns["full_width"].append(el)
            continue
            
        (x1, y1), _, (x2, y2), _ = coords.get("points")
        el_width = x2 - x1
        el_center_x = (x1 + x2) / 2

        if el_width > page_width * full_width_threshold:
            columns["full_width"].append(el)
        elif el_center_x < page_midpoint:
            columns["left_column"].append(el)
        else:
            columns["right_column"].append(el)
            
    # Sort each column by vertical position
    for key in columns:
        columns[key].sort(key=lambda item: item.get("metadata", {}).get("coordinates", {}).get("points", [[0,0]])[0][1])

    return columns

def add_elements_to_cell(cell, elements, doc):
    """Helper to add a list of processed elements to a table cell with robust string casting."""
    for element in elements:
        # THE FIX IS HERE: Use str() to prevent errors from non-string data
        text_to_add = str(element.get('translated_text', ''))
        p = cell.add_paragraph(text_to_add)
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        cell.add_paragraph('') # Add a space between elements

def create_review_document_with_columns(filename: str, paged_translated_elements: Dict[int, List[dict]]):
    """
    Creates a Word document that reconstructs the two-column layout using a table.
    This version includes robust string casting to prevent errors.
    """
    print("--- 📄 Reconstructing Document with Two-Column Layout ---")
    doc = Document()
    doc.sections[0].left_margin = Inches(0.5)
    doc.sections[0].right_margin = Inches(0.5)

    for page_num, elements in sorted(paged_translated_elements.items()):
        page_width = 595 # A4 width in points
        grouped_cols = group_elements_by_column(elements, page_width)
        
        # THE FIX IS HERE: Use str() to prevent errors
        doc.add_heading(f"Page {page_num}", level=1)

        for el in grouped_cols["full_width"]:
            # THE FIX IS HERE: Use str() to prevent errors
            text_to_add = str(el.get('translated_text', ''))
            p = doc.add_paragraph(text_to_add)
            if el.get('type') == 'Title':
                p.style = 'Heading 2'
            doc.add_paragraph('') # Spacer

        if grouped_cols["left_column"] or grouped_cols["right_column"]:
            table = doc.add_table(rows=1, cols=2)
            table.autofit = False
            table.allow_autofit = False
            table.style = 'Table Grid'
            
            table.columns[0].width = Inches(3.75)
            table.columns[1].width = Inches(3.75)
            
            left_cell = table.cell(0, 0)
            right_cell = table.cell(0, 1)
            
            add_elements_to_cell(left_cell, grouped_cols["left_column"], doc)
            add_elements_to_cell(right_cell, grouped_cols["right_column"], doc)

        if page_num < len(paged_translated_elements):
            doc.add_page_break()
            
    doc.save(filename)
    print(f"\nLayout-aware review document saved to '{filename}'")


# ==============================================================================
# 10. MAIN EXECUTION BLOCK (MODIFIED FOR LAYOUT-AWARE PROCESSING)
# ==============================================================================
if __name__ == "__main__":
    # Ensure you copy the unchanged functions here:
    # build_graph(), process_element_task(), etc. from the previous final code.
    # The only change is how we call them.

    app = build_graph() # Unchanged
    doc_manager = DocumentContextManager() # Unchanged
    
    try:
        # Step 1: Parse the document page by page
        paged_source_elements = document_parser(ENGLISH_PDF_PATH)
        
        all_elements_to_process = []
        for page_num, elements in paged_source_elements.items():
            # Flatten the list for the translation pool, but keep page info if needed
            for el in elements:
                el['page_num'] = page_num # Tag element with its page number
                all_elements_to_process.append(el)
        
        elements_to_process = all_elements_to_process[:MAX_ELEMENTS_TO_PROCESS]
        print(f"\n--- 🚀 STARTING TRANSLATION FOR {len(elements_to_process)} ELEMENTS (Workers: {NUM_WORKERS}) ---")

        # Step 2: Translate all elements in parallel (the engine is layout-agnostic)
        tasks = [(i, element, app, doc_manager) for i, element in enumerate(elements_to_process)]
        results_map = {}
        with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
            # The worker function `process_element_task` is unchanged
            future_to_task = {executor.submit(process_element_task, task): task for task in tasks}
            for future in as_completed(future_to_task):
                # We get the original element back to preserve its metadata
                original_task_args = future_to_task[future]
                original_element = original_task_args[1]
                
                _, translated_element_data = future.result()
                
                # Merge the translation result back into the original element data
                original_element.update(translated_element_data)
                results_map[id(original_element)] = original_element

        # Step 3: Re-group the translated elements by page for reconstruction
        paged_translated_elements = {}
        for element in all_elements_to_process:
            if id(element) in results_map:
                page_num = element['page_num']
                if page_num not in paged_translated_elements:
                    paged_translated_elements[page_num] = []
                paged_translated_elements[page_num].append(results_map[id(element)])

        print("\n\n--- ✨ WORKFLOW COMPLETE ---")

        # Step 4: Reconstruct the final document with the two-column layout
        output_file_docx = f"review_document_layout_aware_{str(uuid.uuid4())[:8]}.docx"
        create_review_document_with_columns(output_file_docx, paged_translated_elements)

    except Exception as e:
        print(f"\n--- ❌ A FATAL ERROR OCCURRED ---")
        import traceback
        traceback.print_exc()


LangGraph workflow built successfully with all advanced features.
Thread-safe Document Context Manager initialized.
--- 📄 PARSING DOCUMENT (Layout-Aware) ---
Document parsed into 3 pages.

--- 🚀 STARTING TRANSLATION FOR 15 ELEMENTS (Workers: 1) ---
Processing element 1 | Type: 'Header'
Processing element 2 | Type: 'Title'

--- 📝 DRAFT TRANSLATOR (FinTranslate Specialist) ---
Draft: 'VERMÖGENSWERTES TATSÄCHLICHES VERHÄLTNIS...'
--- 🧐 MASTER CRITIQUE AGENT ---
--- Critiques Found: ---
  - Glossary Violation: 'Sustainability Statement' should be 'Nachhaltigkeitserklärung'.
  - Stylistic Violation: The phrase "VERMÖGENSWERTES TATSÄCHLICHES VERHÄLTNIS" is awkwardly phrased, as it contains a redundant adjective "werts" which is not necessary in this context.
--- 🛠️ LLM REVISER AGENT (Llama-3.1 Generalist) ---
Revised Translation: 'Nachhaltigkeitserklärung...'
Reviser proposed new term: Sustainability Statement -> Nachhaltigkeitserklärung
--- ✅ FINAL RECONSTRUCTOR & MEMORY UPDATE ---
CONTEXT 